# Using `ctypes` to Interface C with Python

In this notebook, we'll explore how to use Python's `ctypes` library to interface with a C function that fills a NumPy array.

## Prerequisites

Before we begin, ensure you have the following:

- Python 3.x installed
- GCC compiler (for Linux/macOS) or MinGW (for Windows)
- NumPy library installed
- Basic understanding of C and Python programming

## 1. Writing the C Code

Create a file named `array_fill.c` with the following content:

In [ ]:
%%writefile array_fill.c

#include <stddef.h>

void fill_array(double *arr, size_t size) {
    for (size_t i = 0; i < size; ++i) {
        arr[i] = (double)i;
    }
}

## 2. Compiling the C Code into a Shared Library

Compile the C code to create a shared library that can be loaded by Python.

In [ ]:
!gcc -shared -o libarrayfill.so -fPIC array_fill.c

## 3. Creating the Python Wrapper

We'll use `ctypes` to load the shared library and define a Python function that wraps the C function.

In [ ]:
import ctypes
import numpy as np
import os

# Determine the path to the shared library
libname = './libarrayfill.so'

# Load the shared library
lib = ctypes.CDLL(libname)

# Define the argument types for the C function
lib.fill_array.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t
]
lib.fill_array.restype = None

def fill_array(arr):
    arr = np.ascontiguousarray(arr, dtype=np.float64)
    size = arr.size
    lib.fill_array(arr, size)

## 4. Using the Wrapper Function

Let's create a NumPy array and use the `fill_array` function to fill it.

In [ ]:
# Create a NumPy array of zeros
array = np.zeros(10, dtype=np.float64)

# Use the C function to fill the array
fill_array(array)

print("Filled Array:")
print(array)

## 5. Extending to Two-Dimensional Arrays

We can modify the C function and the wrapper to work with 2D arrays.

In [ ]:
%%writefile array_fill_2d.c

#include <stddef.h>

void fill_array_2d(double *arr, size_t rows, size_t cols) {
    for (size_t i = 0; i < rows; ++i) {
        for (size_t j = 0; j < cols; ++j) {
            arr[i * cols + j] = (double)(i * cols + j);
        }
    }
}

In [ ]:
!gcc -shared -o libarrayfill2d.so -fPIC array_fill_2d.c

In [ ]:
# Load the new shared library
lib2d = ctypes.CDLL('./libarrayfill2d.so')

# Define the argument types
lib2d.fill_array_2d.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=2, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_size_t
]
lib2d.fill_array_2d.restype = None

def fill_array_2d(arr):
    arr = np.ascontiguousarray(arr, dtype=np.float64)
    rows, cols = arr.shape
    lib2d.fill_array_2d(arr, rows, cols)

In [ ]:
# Create a 2D NumPy array
array_2d = np.zeros((3, 4), dtype=np.float64)

# Use the C function to fill the array
fill_array_2d(array_2d)

print("Filled 2D Array:")
print(array_2d)

## 6. Conclusion

We've successfully used `ctypes` to interface with C functions and manipulate NumPy arrays directly from C.